# Bayesian Analysis and Theory
# Project 2

**Naman Chhibbar, MA21BTECH11011** \
**Abhay Gupta, BM21BTECH11001**

## Installing and Importing Dependencies

In [1]:
# Uncomment and run the following line to install the required packages
# !pip install -qU numpy pandas matplotlib

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Importing Data

In [3]:
data_path = 'combined_data.csv'

df = pd.read_csv(data_path)
df.tail()

,Product,Quantity Ordered,Total Revenue,Month
223,ThinkPad Laptop,541,540994.59,December
224,USB-C Charging Cable,3251,38849.45,December
225,Vareebadd Phone,285,114000.00,December
226,Wired Headphones,2748,32948.52,December
227,iPhone,908,635600.00,December


In [4]:
# Extract columns of interest
data = df[['Product', 'Total Revenue', 'Month']]
data.tail()

,Product,Total Revenue,Month
223,ThinkPad Laptop,540994.59,December
224,USB-C Charging Cable,38849.45,December
225,Vareebadd Phone,114000.00,December
226,Wired Headphones,32948.52,December
227,iPhone,635600.00,December


In [5]:
# Choose a random product
products = data['Product'].unique()
product = np.random.choice(products)
product

'Apple Airpods Headphones'

In [7]:
# Extract product data
filt = data['Product'] == product
product_data = data[filt][['Total Revenue', 'Month']]
product_data.tail()

,Total Revenue,Month
139,151500.0,August
158,145200.0,September
177,251100.0,October
196,226500.0,November
215,311850.0,December
